# Imports

* Importy PyTorch: torch, torch.nn i torch.nn.functional używane do podstawowych operacji tensorowych i modułów sieci neuronowych.
* Import math służy do normalizacji pierwiastka kwadratowego w attention.
* Einops rearrange służy do przekształcania i permutacji tensorów w przyjazny dla czytelnika i wydajny sposób.





In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from einops import rearrange
from torchvision.ops.stochastic_depth import StochasticDepth # Add stochastic depth

# Patch Partition + Linear Embedding

---
„Najpierw dzieli wejściowy obraz RGB na nienakładające się patchs za pomocą modułu dzielenia patch, takiego jak ViT. Każda patch jest traktowana jako „token”, a jej cecha jest ustawiana jako konkatenacja surowych wartości RGB pikseli. W naszej implementacji używamy patch o rozmiarze 4 × 4, a zatem wymiar funkcji każdego patcha wynosi 4 × 4 × 3 = 48. Liniowa warstwa osadzania jest stosowana na tej surowej funkcji, aby rzutować ją na dowolny wymiar (oznaczony jako C)”.

---

Gdzie C jest hyperparametrem, który określa wymiar osadzenia. W naszym przypadku C = 96, dla modelu Swin-Transformer(tiny).




![image](../images/Patch_Partition_Linear_Embedding.png)


Podział patchy w stylu ViT i liniowe embeding można zrealizować za pomocą splotu z rozmiarem jądra, krokiem (stride) równym rozmiarowi patcha oraz wyjściowymi kanałami równymi \(C\). Wynikowy tensor ma wymiary \(H/p * W/p * C\), gdzie każdy „token” odpowiada liniowemu przekształceniu pikseli patcha. Wymiar embedings \(C\) to liczba cech (kanałów), które opisują każdą jednostkę w reprezentacji danych. W naszym przypadku \(C = 96\).

Klasa **SwinEmbedding**, dziedzicząca z **nn.Module**, inicjalizuje:
1. Warstwę splotu \(p * p\) (stride \(p\)), z kanałami wyjściowymi \(C\),
2. **LayerNorm** dla wymiaru embeding \(C\),
3. Funkcję aktywacji ReLU.

W metodzie `forward` wejście jest przepuszczane przez splot, przekształcane i permutowane, łącząc \(H, W\) w \(H * W / p^2\), a wymiar osadzania \(C\) przesuwany na końcową pozycję. Na końcu stosowane są normalizacja i ReLU.

In [11]:
class SwinEmbedding(nn.Module):

  """
  input shape -> (b,c,h,w)
  output shape -> (b, (h/4 * w/4), C)

  Where:

  b - batch size
  c - number of channels
  h - height of the image
  w - width of the image
  C - number of channels in the output

  """

  def __init__(self, patch_size = 4, C = 96):
      super().__init__()
      self.linear_embedding = nn.Conv2d(3,C, kernel_size=patch_size, stride=patch_size)
      self.layer_norm = nn.LayerNorm(C)
      self.relu = nn.ReLU()

  
  def forward(self,x):
    x = self.linear_embedding(x)
    x = rearrange(x, 'b c h w -> b (h w) c')  # spłaszczenie wymiarów przestrzennych obrazu przy pomocy mnożenia h i w
    x = self.layer_norm(x) # normalizacja
    x = self.relu(x) # funkcja aktywacji (dodanie nieliniowości)

    return x



# Patch Merging Layer

![image](../images/hearachical_system.png)

Aby stworzyć hierarchiczną reprezentację, liczba tokenów jest zmniejszana przez warstwy scalania patchy, gdy sieć staje się głębsza. Pierwsza warstwa scalania patch'y łączy cechy każdej grupy 2 × 2 sąsiednich patch'y i stosuje warstwę liniową na 4C-wymiarowych połączonych cechach. Zmniejsza to liczbę tokenów o wielokrotność 2×2 = 4 (2-krotne zmniejszenie rozdzielczości), a wymiar wyjściowy jest ustawiony na 2C.
Inicjalizujemy warstwę liniową z kanałami wejściowymi 4C do kanałów wyjściowych 2C i inicjalizujemy normę warstwy z wyjściowym rozmiarem osadzania. W naszej funkcji forward używamy einops rearrange do zmiany kształtu naszych tokenów z 2x2xC na 1x1x4C. Kończymy, przepuszczając nasze dane wejściowe przez projekcję liniową i normę warstwy.

![image](../images/Patch_mergering.png)


In [12]:
class PatchMerging(nn.Module):

  """
  Reduces tokens by a factor of 4 (2x2 patches) and doubles embedding dimension.


  input shape -> (b (h w) c)
  output shape -> (b (h/2 * w/2) C*2)

  Where:

  b - batch size
  c - number of channels
  h - height of the image
  w - width of the image

  """

  def __init__(self, C) -> None:
     super().__init__()
     self.linear_layer = nn.Linear(C*4, C*2) # podwajamy wymiar embeddingów
     self.layer_norm = nn.LayerNorm(2 * C) # normalizacja

  def forward(self, x):
    height = width = int(math.sqrt(x.shape[1])/ 2) # obliczamy nową wysokość i szerokość obrazu
    x = rearrange(x, 'b (h s1 w s2) c -> b (h w) (s2 s1 c)', s1=2, s2=2, h=height, w=width)
    x = self.linear_layer(x)
    x = self.layer_norm(x)
    return x

# Shifted Window Attention Mechanism


Zaczynamy od zainicjowania naszych parametrów embed_dim, num_heads i window_size oraz zdefiniowania dwóch projekcji liniowych. Pierwsza z nich to nasza projekcja z danych wejściowych do zapytań, kluczy i wartości, którą wykonujemy w jednej równoległej projekcji, więc rozmiar wyjściowy jest ustawiony na 3*C. Druga projekcja to projekcja liniowa zastosowana po obliczeniach uwagi. Projekcja ta służy do komunikacji między połączonymi równoległymi wielogłowicowymi jednostkami uwagi.

Rozpoczynamy naszą funkcję do przodu, uzyskując rozmiar naszej głowy, wysokość i szerokość naszego wejścia, ponieważ potrzebujemy tych parametrów do zmiany układu. Następnie wykonujemy projekcję Q,K,V na naszym wejściu o kształcie ((h*w), c) do ((h*w), 3C). Nasz następny krok składa się z dwóch części, w których zmienimy nasze dane wejściowe ((h*w), C*3) na okna i równoległe głowice uwagi do naszych obliczeń uwagi.

Pózniej rozbijamy naszą macierz na 3 macierze Q,K,V i obliczamy uwagę za pomocą standardowego wzoru uwagi:

Formuła self-attention w mechanizmie transformera wygląda następująco:

$$
\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^\top}{\sqrt{d_k}}\right)V
$$



**Obliczanie Attention Scores**

```python
attention_scores = (Q @ K.transpose(4, 5)) / math.sqrt(h_dim)
```

Dla każdego tokena obliczamy podobieństwo (iloczyn skalarny) między wektorem zapytania  $Q$ a wszystkimi kluczami $K$. Następnie dzielimy przez $(\sqrt{d_k})$, aby zachować stabilność gradientów. Pózniej wyniki zmarnalizowane poprzez  dzielienia na $d_k$, gdzie $d_k$ to wymiar wektorów $Q$ i $K$. Dzielimy przez $(\sqrt{d_k})$, aby zachować stabilność gradientów.


**Softmax i kontekst uwagi**

```python
attention = F.softmax(attention_scores, dim=-1) @ V
```

Obliczamy softmax z $( \text{attention\_scores} )$ w celu uzyskania prawdopodobieństw, które określają „na co” dany token zwraca uwagę. Następnie obliczamy „ważoną sumę” wartości $V$ na podstawie macierzy uwagi. Wynikiem jest nowa reprezentacja każdego tokena, wzbogacona o informacje z innych tokenów w oknie.


Ze względu na sposób, w jaki ukształtowaliśmy nasze macierze, obliczenia uwagi w oknach są wykonywane wydajnie równolegle w oknach i głowicach uwagi. Na koniec przestawiamy tensory z powrotem na ((h*w),C) i zwracamy nasze ostateczne przewidywane dane wejściowe.

![image.png](../images/self-attetention.png)

Później wprowadzamy **Shifted Window Attention Mechanism** w Swin Transformerach umożliwiający wymianę informacji między nieprzecinającymi się okienkami poprzez wprowadzenie przesunięcia ich układu w kolejnych warstwach. Przesunięcie to sprawia, że sąsiednie okienka częściowo na siebie nachodzą, co pozwala na przepływ informacji przez ich granice. Przesunięcie jest realizowane wydajnie za pomocą operacji cyklicznej (np. `torch.roll`), która przemieszcza okienka o połowę ich rozmiaru.

Wyzwanie pojawia się w związku z przesunięciem, ponieważ tokeny z różnych okienek mogą zostać przestrzennie źle dopasowane. Aby temu zapobiec, stosuje się maskowanie uwagi, które blokuje interakcje między tokenami nienależącymi do sąsiednich obszarów obrazu. Maski te są zaprojektowane tak, aby uniemożliwić uwzględnianie informacji między regionami niepołączonymi w oryginalnym układzie.

Ten mechanizm nie tylko umożliwia lokalną uwagę w obrębie okienek, ale także wspiera hierarchiczne uczenie cech poprzez tworzenie połączeń między sąsiadującymi okienkami w kolejnych warstwach.

![image.png](../images/shifted_window_attention_mechanism.webp)

In [13]:
class ShiftedWindowMSA(nn.Module):

    """
    input shape -> (b , (h*w), C)
    output shape -> (b , (h*w), C)

    Where:

    b - batch size
    h - height of the image
    w - width of the image
    C - number of channels in the output
    """
      
    def __init__(self, embed_dim, num_heads, window_size=7, mask=False):
        super().__init__()
        self.embed_dim = embed_dim # wymiar embeddingów
        self.num_heads = num_heads # liczba głów
        self.window_size = window_size # rozmiar okna
        self.mask = mask # maska (True/False)
        self.proj1 = nn.Linear(embed_dim, 3*embed_dim) # projekcja wejścia
        self.proj2 = nn.Linear(embed_dim, embed_dim) # projekcja wyjścia
        self.embeddings = RelativeEmbeddings() 

    def forward(self, x):
        h_dim = self.embed_dim / self.num_heads # obliczamy wymiar pojedynczej głowy
        height = width = int(math.sqrt(x.shape[1])) 
        x = self.proj1(x) 
        x = rearrange(x, 'b (h w) (c K) -> b h w c K', K=3, h=height, w=width) # zmiana wymiarów, gdzie K to liczba macierzy Q,K,V
 
        if self.mask: # jeśli maska jest True, to wykonujemy przesunięcie okna o połowę
            x = torch.roll(x, (-self.window_size//2, -self.window_size//2), dims=(1,2))

        # zmiana wymiarów
        x = rearrange(x, 'b (h m1) (w m2) (H E) K -> b H h w (m1 m2) E K', H=self.num_heads, m1=self.window_size, m2=self.window_size)
       
        # podział na macierze Q,K,V
        Q, K, V = x.chunk(3, dim=6)  # dzielimy na 3 części
        Q, K, V = Q.squeeze(-1), K.squeeze(-1), V.squeeze(-1) # usuwamy ostatni wymiar, bo nie jest potrzebny
        attention_scores = (Q @ K.transpose(4,5)) / math.sqrt(h_dim) # obliczamy self-attention score
        attention_scores = self.embeddings(attention_scores) # dodajemy embeddingsy

        '''
        H - attention heads 
        h,w - vertical and horizontal dimensions of the image
        (m1 m2) - total size of the window
        E - head dimension
        K = 3 - constant to break our matrix into 3 Q,K,V matricies
      
        shape of attention_scores = (b, H, h, w, (m1*m2), (m1*m2))
        we simply have to generate our row/column masks and apply them
        to the last row and columns of windows which are [:,:,-1,:] and [:,:,:,-1]
        
        '''

        if self.mask: # jeśli maska jest True, to wykonujemy maskowanie ostatnich wierszy i kolumn w oknie 
            row_mask = torch.zeros((self.window_size**2, self.window_size**2)).cuda() # tworzymy maskę
            row_mask[-self.window_size * (self.window_size//2):, 0:-self.window_size * (self.window_size//2)] = float('-inf') 
            row_mask[0:-self.window_size * (self.window_size//2), -self.window_size * (self.window_size//2):] = float('-inf')
            column_mask = rearrange(row_mask, '(r w1) (c w2) -> (w1 r) (w2 c)', w1=self.window_size, w2=self.window_size).cuda() # maska kolumn 
            attention_scores[:, :, -1, :] += row_mask 
            attention_scores[:, :, :, -1] += column_mask

        attention = F.softmax(attention_scores, dim=-1) @ V # Softmax i mnożenie przez V 
        x = rearrange(attention, 'b H h w (m1 m2) E -> b (h m1) (w m2) (H E)', m1=self.window_size, m2=self.window_size)

        if self.mask: # Z powrotem przesuwamy okno o połowę
            x = torch.roll(x, (self.window_size//2, self.window_size//2), (1,2))

        x = rearrange(x, 'b h w c -> b (h w) c')
        return self.proj2(x) # projekcja wyjścia


# Relative Position Embeddings

**Relative Position Embeddings**  wprowadzają dodatkową macierz biasu do mechanizmu uwagi własnej, aby uwzględnić relacje przestrzenne między tokenami. W obliczeniach uwagi macierz biasu pozycyjnego $B \in \mathbb{R}^{M^2 \times M^2}$ jest dodawana do wyników podobieństwa, co pozwala modelowi lepiej rozumieć strukturę przestrzenną tokenów w obrębie okienka.

Aby zmniejszyć złożoność, pełna macierz $B$ jest generowana z mniejszej macierzy parametrów $\hat{B} \in \mathbb{R}^{(2M-1) \times (2M-1)}$, gdzie $M$ to rozmiar okienka. Wartości w $B$ są wypełniane na podstawie względnych pozycji tokenów w zakresie $[-M+1, M-1]$ wzdłuż każdej osi. Później macierz $B$ jest dzielona na 4 podmacierze, które są dodawane do wyników podobieństwa w celu uwzględnienia relacji przestrzennych w pionie, poziomie i obu kierunkach przekątnych. Modyfikacja tej formuły wygląda następująco:

$$
\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^\top}{\sqrt{d_k}}\right)V + B
$$



W odróżnieniu od osadzeń sinusoidalnych, te osadzenia pozycyjne są uczone podczas treningu, co daje większą elastyczność. Dzięki dodaniu tych osadzeń bezpośrednio do wyników iloczynu zapytań i kluczy, model efektywnie uwzględnia informacje o relacjach przestrzennych, zachowując jednocześnie zgodność wymiarów w obliczeniach uwagi. To rozwiązanie umożliwia lepsze odwzorowanie relacji przestrzennych przy użyciu zwartej reprezentacji parametrów.

![image.png](../images/relative_position_embeddings.png)



In [14]:
class RelativeEmbeddings(nn.Module):
    def __init__(self, window_size=7):
        super().__init__()
        B = nn.Parameter(torch.randn(2*window_size-1, 2*window_size-1))
        x = torch.arange(1,window_size+1,1/window_size)
        x = (x[None, :]-x[:, None]).int()
        y = torch.concat([torch.arange(1,window_size+1)] * window_size)
        y = (y[None, :]-y[:, None])
        self.embeddings = nn.Parameter((B[x[:,:], y[:,:]]), requires_grad=False)

    def forward(self, x):
        return x + self.embeddings

# Transformer Encoder Block

**Transformer Encoder Block** w Swin Transformer jest zgodny z typową architekturą bloku transformera, z tą różnicą, że wykorzystuje mechanizm uwagi w przesuniętych oknach oraz aktywację GELU w wielowarstwowej perceptronie (MLP). Każdy blok kodera składa się z dwóch głównych etapów: obliczania uwagi oraz przekształceń nieliniowych w MLP.

W pierwszym etapie dane wejściowe są normalizowane i przekazywane do mechanizmu uwagi z przesuniętymi oknami (Shifted Window Attention). Mechanizm ten umożliwia komunikację między sąsiednimi oknami, a wynik uwagi jest dodawany jako rezidual do oryginalnych danych. 

Następnie dane przechodzą przez drugi etap, który obejmuje normalizację, warstwę MLP rozszerzającą wymiar przestrzeni osadzania czterokrotnie, zastosowanie aktywacji GELU oraz powrót do pierwotnego wymiaru. Wynik jest również sumowany z danymi z poprzedniego etapu, co umożliwia lepsze propagowanie informacji w sieci.

Dodatkowo wprowadzono klasę **AlternatingEncoderBlock**, która grupuje bloki kodera w pary. Pierwszy blok w parze działa na standardowych oknach, a drugi na przesuniętych oknach, co pozwala na efektywne uchwycenie relacji między tokenami w różnych lokalizacjach.

![image.png](../images/transformer_encoder_block.webp)

In [15]:
class SwinEncoderBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, window_size, mask, sd_prob=0.1):
        super().__init__()
        self.layer_norm = nn.LayerNorm(embed_dim)
        self.stochastic_depth = StochasticDepth(sd_prob, "row") # Stochastic Depth with 0.1 probability of dropping out a row for tiny version of Swin Transformer

        self.WMSA = ShiftedWindowMSA(embed_dim=embed_dim, num_heads=num_heads, window_size=window_size, mask=mask)
        self.MLP = nn.Sequential(
            nn.Linear(embed_dim, embed_dim*4),
            nn.GELU(),
            nn.Dropout(p=0.1), # Default dropout probability is 0.0 in the torchvision implementation
            nn.Linear(embed_dim*4, embed_dim)
        )

        # Initialization of weights and biases (bias) in linear layers 
        for m in self.MLP:
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight) # Xavier initialization for weights, which prevents the disappearance or explosion of gradients during training.
                if m.bias is not None:
                    nn.init.normal_(m.bias, std=1e-6) # Set a small offset, to have a small impact in the initial stages of training.

    def forward(self, x):
        
        # Attention path with pre-normalization 
        res1 = x # Save input for the skip connection
        x = self.stochastic_depth(self.WMSA(self.layer_norm(x))) # Attention block with LayerNorm and Stochastic Depth(more efficient than Dropout for training transformers)
        x = res1 + x # Residual connection

        # MLP path with pre-normalization
        res2 = x  # Save intermediate result for skip connection
        x = self.stochastic_depth(self.MLP(self.layer_norm(x))) # MLP block with LayerNorm and Dropout
        x = res2 + x  # Residual connection

        return x
    
class AlternatingEncoderBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, sd_prob, window_size=7):
        super().__init__()
        self.WSA = SwinEncoderBlock(embed_dim=embed_dim, num_heads=num_heads, window_size=window_size, mask=False, sd_prob=sd_prob[0])
        self.SWSA = SwinEncoderBlock(embed_dim=embed_dim, num_heads=num_heads, window_size=window_size, mask=True, sd_prob=sd_prob[1])
    
    def forward(self, x):
        return self.SWSA(self.WSA(x))

# Final Swin-Transformer Class

Mając już zaimplementowane wszystkie komponenty Swin-Transformera, możemy stworzyć jego finalną klasę. Struktura modelu opiera się na oryginalnym artykule, uwzględniając odpowiednie bloki kodera, wymiary osadzeń oraz liczbę głów uwagi.

Model zaczyna się od warstwy osadzania (*Embedding Layer*), która przekształca obraz wejściowy w odpowiednią reprezentację. Następnie przechodzi przez cztery etapy obliczeniowe:  
1. **Etap 1**: Alternating Encoder Block z 96 wymiarami osadzania i 3 głowami uwagi.  
2. **Etap 2**: Alternating Encoder Block z 192 wymiarami osadzania i 6 głowami uwagi.  
3. **Etap 3**: Trzy następujące po sobie Alternating Encoder Blocks z 384 wymiarami osadzania i 12 głowami uwagi.  
4. **Etap 4**: Alternating Encoder Block z 768 wymiarami osadzania i 24 głowami uwagi.  

Każdy etap zawiera proces *Patch Merging*, który zmniejsza rozdzielczość przestrzenną danych i zwiększa liczbę wymiarów kanałów. Finalnie, dane wyjściowe mają wymiary `(1, 49, 768)`, gdzie 1 to wymiar partii, 49 to spłaszczona przestrzeń 7x7, a 768 to liczba kanałów reprezentująca wymiar osadzania.

Testując model z obrazem wejściowym o wymiarach `(1, 3, 224, 224)`, możemy potwierdzić, że implementacja działa zgodnie z oczekiwaniami i generuje poprawne dane wyjściowe. Dzięki temu w pełni zaimplementowaliśmy Swin-Transformer w PyTorch od podstaw!

![image.png](../images/all_stages_swin.png)

In [16]:
class SwinTransformer(nn.Module):
    def __init__(self, depth=[2, 2, 6, 2], embed_dim=96, stochastic_depth_prob=0.2):
        super().__init__()
        self.Embedding = SwinEmbedding()  # Embedding layer

        # Calculate total number of blocks
        total_stage_blocks = sum(depth)
        stage_block_id = 0

        self.stages = nn.ModuleList()

        in_channels = embed_dim
        for i_stage, num_blocks in enumerate(depth):
            temp_sd_prob = []
            for _ in range(num_blocks):
                # Calculate probability for the current layer
                sd_prob = stochastic_depth_prob * float(stage_block_id) / (total_stage_blocks - 1)
                temp_sd_prob.append(sd_prob)
                stage_block_id += 1

            #Add alternating encoder blocks recording to the depth list divided by 2, because each block has 2 sub-blocks
            sd_prob = [temp_sd_prob[i:i+2] for i in range(0, len(temp_sd_prob), 2)]
            for _ in range(int(num_blocks / 2)):
                num_heads = in_channels // 32
                #print(f"AlternatingEncoderBlock({in_channels}, {num_heads}, {sd_prob[0]})") # Debug
                self.stages.append(
                    AlternatingEncoderBlock(in_channels, num_heads, sd_prob[0])
                )
                sd_prob.pop(0)
                    
            # Add patch merging layer if this is not the last stage
            if i_stage < len(depth) - 1:
                self.stages.append(PatchMerging(in_channels))
                #print(f"PatchMerging({in_channels})") # Debug
                in_channels *= 2

    def forward(self, x):
        x = self.Embedding(x) 
        for stage in self.stages: 
            x = stage(x)
        
        return x


In [17]:
def main():
    x = torch.randn((1,3,224,224)).cuda()
    model = SwinTransformer().cuda()
    print(model(x).shape)

if __name__ == '__main__':
    main()

torch.Size([1, 49, 768])
